### Process-Mining-Predictive-Analysis-using-Python

# Importing and displaying petrinet 

In [3]:
import os
import pm4py

from pm4py.objects.petri.importer import importer as pnml_importer

process_model, initial_marking, final_marking = pnml_importer.apply("../assets/ProcessModel.pnml")


from pm4py.visualization.petrinet import visualizer as pn_visualizer
gviz = pn_visualizer.apply(process_model, initial_marking, final_marking)
pn_visualizer.view(gviz)
print(process_model)

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tpng', '-O', 'tmp26uq3d4a.gv'], make sure the Graphviz executables are on your systems' PATH

In [ ]:
places = process_model.places
transitions = process_model.transitions
arcs = process_model.arcs

## Importing log file

In [ ]:
from pm4py.objects.log.importer.xes import importer as xes_importer
log = xes_importer.apply('../assets/BPI_Challenge_2017.xes')

## Creating sampled_log from the log file


In [ ]:
from pm4py.objects.log.util import sampling
sampled_log = sampling.sample(log, n=1000)

## Creating dataframe from log file


In [ ]:
import pandas as pd

df = pd.DataFrame (columns = ['process_id','event_id' ,'event_name'])

for case_index, case in enumerate(sampled_log):
    for event_index, event in enumerate(case):
        df = df.append(pd.DataFrame({'process_id': case_index, 'event_id': event_index, 'event_name': event["concept:name"]}, index=[0]), ignore_index=True)
        
df2 = df.groupby(['process_id'])['event_name'].apply(list).reset_index()

## Getting next possible tasks from current task

In [ ]:
def is_Sublist(l, s):
    sub_set = False
    next_activity=None
    if s == []:
        sub_set = True
    elif s == l:
        sub_set = True
    elif len(s) > len(l):
        sub_set = False

    else:
        for i in range(len(l)):
            if l[i] == s[0]:
                n = 1
                while (n < len(s)) and (l[i+n] == s[n]):
                    n += 1
                if n == len(s):
                    sub_set = True
                    next_activity=l[n+i]

    return sub_set, next_activity

In [ ]:
S = ['A_Create Application', 'A_Concept'] #Incomplete process input

current_task = S[-1] #Last element is current task

next_transitions = {} # holding ngram occurance count in the dict 

for current_tran in transitions: #Looking through all transitions in the model
    if str(current_tran) == current_task:
        for out_arc_current in current_tran.out_arcs: #Getting out arcs of current transition
            for arc2 in out_arc_current.target.out_arcs: #Looping out arcs of next place
                next_transitions[arc2.target.label]=0 #Getting next transition

next_transitions.pop(None, None)
next_transitions

#n-grams
n_grams = [S[i:] for i in range(len(S))]

#n-gram probability dataframe
prob_df = pd.DataFrame (columns = next_transitions)

prob_total=0    

for n_gram in n_grams:
    for index, case in df2.iterrows():
        is_sub_set, next_activity=is_Sublist(case['event_name'],n_gram)
        if next_activity in list(next_transitions.keys()):
            next_transitions[next_activity]=next_transitions[next_activity]+1
            prob_total=prob_total+1

for index, x in enumerate(n_grams):
    prob_df = prob_df.append(pd.Series(0, index=prob_df.columns), ignore_index=True)
    for key,value in next_transitions.items():
        if(prob_total>0):
            prob_df.iloc[index][key]=value/prob_total

# JUST IN CASE IF THERE ARE NO NGRAMS THEN ADD SINGLE ROW OF ALLL ZEROS            
if(len(prob_df)==0):
    prob_df = prob_df.append(pd.Series(0, index=prob_df.columns), ignore_index=True)  

prob_df.index = prob_df.index + 1

# Final Probabilities

In [ ]:
prob_df